In [2]:
from google.colab import files
uploaded = files.upload()

Saving BCH-USD.csv to BCH-USD.csv
Saving BTC-USD.csv to BTC-USD.csv
Saving ETH-USD.csv to ETH-USD.csv
Saving LTC-USD.csv to LTC-USD.csv


In [12]:
!mkdir models
!ls

mkdir: cannot create directory ‘models’: File exists
BCH-USD.csv  BTC-USD.csv  ETH-USD.csv  LTC-USD.csv  models  sample_data


#Importing dataset


Importing the libraries

In [0]:
import pandas as pd

Importing the data from the files

In [14]:
pd.read_csv('BCH-USD.csv',names=['time','low','high','open','close','volume'])

,time,low,high,open,close,volume
0,1528968660,871.650024,871.729980,871.650024,871.719971,5.675361
1,1528968720,870.859985,871.719971,871.719971,870.859985,26.856577
2,1528968780,870.099976,871.090027,871.090027,870.099976,1.124300
3,1528968840,868.830017,870.950012,868.830017,870.789978,1.749862
4,1528968900,870.000000,870.000000,870.000000,870.000000,1.680500
...,...,...,...,...,...,...
92220,1535215020,531.479980,531.479980,531.479980,531.479980,0.016868
92221,1535215080,531.469971,531.469971,531.469971,531.469971,0.013854
92222,1535215140,531.479980,531.479980,531.479980,531.479980,0.016900
92223,1535215200,531.479980,531.479980,531.479980,531.479980,0.299520


Making the final dataframe.The final dataframe contains all the close and volume values from the all the csv files merged together with the timestamp as the index.

In [0]:
data=['BCH-USD','BTC-USD','ETH-USD','LTC-USD']

In [16]:
main_df=pd.DataFrame()
for dataset in data:
  d=f'{dataset}.csv'
  df=pd.read_csv(d,names=['time','low','high','open','close','volume'])
  #print(df.head(5))
  df.rename(columns={"close":f'{dataset}_close','volume':f"{dataset}_volume"},inplace=True)
  df.set_index("time",inplace=True)
  df=df[[f'{dataset}_close',f"{dataset}_volume"]]
  
  if len(main_df)==0:
    main_df=df
  else:
    main_df=main_df.join(df)

#printing the head of the main_df
main_df.head(10)

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume
time,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997
1528969020,869.450012,0.865200,6477.220215,2.731950,485.98999,85.877251,96.440002,95.524078
1528969080,869.989990,23.534929,6480.000000,2.174240,485.98999,160.915192,96.470001,175.205307
1528969140,870.000000,2.300000,6479.990234,0.903100,485.98999,61.371887,96.400002,43.652802


#Data Preprocessing 

###Setting up the initial parameters and conditions 

**What do we want to do ?**<br>
We want to predict the prices for the future period of time using the current prices of the currency .  

**Initial Setup**
*   We decide a sequence length which is required to train a model,this sequence represents the number of past observations required to make the predictions.(SEQ_LEN)

*   Then , we decide a time in the future for which we want to predict the prices.(FUTURE_PERIOD_PREDICT)

* We choose a column for which we want to predict the prices.(RATIO_TO_PREDICT) 



In [0]:
SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT='LTC-USD'

The function which helps us detemine whether buying a currency is profitable or not in the future .

In [0]:
def classify(current,future):
  if( float(future)> float(current)):
    return 1
  else:
    return 0

Setting up the future and the target column in the main dataframe

In [19]:
main_df['future']=main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)
main_df.head(10)

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume,future
time,,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200,96.500000
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024,96.389999
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799,96.519997
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067,96.440002
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978,96.470001
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997,96.400002
1528969020,869.450012,0.865200,6477.220215,2.731950,485.98999,85.877251,96.440002,95.524078,96.400002
1528969080,869.989990,23.534929,6480.000000,2.174240,485.98999,160.915192,96.470001,175.205307,96.400002
1528969140,870.000000,2.300000,6479.990234,0.903100,485.98999,61.371887,96.400002,43.652802,96.400002


In [20]:
#we are going to make a list by mapping the columns to a function and then assign the list as a column 
main_df['target']=list(map(classify,main_df[f'{RATIO_TO_PREDICT}_close'],main_df['future']))
main_df.head(10)

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume,future,target
time,,,,,,,,,,
1528968660,871.719971,5.675361,6489.549805,0.587100,NaN,NaN,96.580002,9.647200,96.500000,0
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024,96.389999,0
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799,96.519997,0
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067,96.440002,0
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978,96.470001,1
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997,96.400002,0
1528969020,869.450012,0.865200,6477.220215,2.731950,485.98999,85.877251,96.440002,95.524078,96.400002,0
1528969080,869.989990,23.534929,6480.000000,2.174240,485.98999,160.915192,96.470001,175.205307,96.400002,0
1528969140,870.000000,2.300000,6479.990234,0.903100,485.98999,61.371887,96.400002,43.652802,96.400002,0


### Making the validation set and the main dataframe

**We want to carefully seperate the validation data and the training data .In this case getting a random sequence of data and shuffling the data does not help .The validation data has to be an accurate representation of the future**

Sorting rows using the timestamp 

In [0]:
times=sorted(main_df.index.values)

Getting the index of the last 5 percent of the data

In [0]:
last_5pct=times[-int(0.05*len(times))]

In [23]:
last_5pct # timestamp of the row which marks the beginning of the last 5 percent of the data

1534879920

Making the validation set

In [0]:
validation_main_df=main_df[(main_df.index>=last_5pct)]

Making the training set

In [0]:
main_df=main_df[(main_df.index<last_5pct)]

###Preprocessing function

We design a pre-processing function that performs the following operations on the data:


1.   Normalising the data
2.   Scaling the data
3. Building the sequential data
4. Shuffling the resultant sequences
5. Balancing the sequential data 
6. Returning the matrix of features and the dependent variable



In [0]:
from sklearn import preprocessing 
from collections import deque
import random
import numpy as np

def preprocess_df(df):
  df=df.drop('future',1)

#here we are pre-processing the values , normalising and scaling them
  for col in df.columns:
    if col!='target':
      df[col]=df[col].pct_change() #Normalising the data 
      df.dropna(inplace=True)
      df[col]=preprocessing.scale(df[col].values) #scaling
  df.dropna(inplace=True)

#creating sequences
  sequential_data=[]
  prev_days=deque(maxlen=SEQ_LEN) # deque function stops appending new elements as soon as the length equals the maximum length specified 

#building the sequential data
  for i in df.values:
    prev_days.append([n for n in i[:-1]]) # list of values of 60 rows (a sequence), this is the input data for the RNN  
    if len(prev_days)==SEQ_LEN:
      sequential_data.append([np.array(prev_days),i[-1]]) # sequences of 60 each along with their label 
#shuffling the sequences
  random.shuffle(sequential_data)
  
#balancing the data
  buys=[]
  sells=[]
#balancing the data and building again sequential data
  for seq,target in sequential_data:
    if target==0:
      sells.append([seq,target])
    elif target==1:
      buys.append([seq,target])
  
  random.shuffle(buys)
  random.shuffle(sells)
  lower=min(len(buys),len(sells))
  buys=buys[:lower]
  sells=sells[:lower]
  
  sequential_data=buys+sells
  random.shuffle(sequential_data)

#splitting into X,y
  X=[]
  y=[]

  for seq,target in sequential_data:
    X.append(seq)
    y.append(target)
  
  return np.array(X),y

Applying it on the dataframe


In [0]:
X_train,y_train=preprocess_df(main_df)

In [0]:
X_validation,y_validation=preprocess_df(validation_main_df)

Printing out some statistics 

In [29]:
print(f"train data: {len(X_train)} validation: {len(X_validation)}")
print(f"Dont buys: {y_train.count(0)}, buys: {y_train.count(1)}")
print(f"VALIDATION Dont buys: {y_validation.count(0)}, buys: {y_validation.count(1)}")

train data: 69400 validation: 3688
Dont buys: 34700, buys: 34700
VALIDATION Dont buys: 1844, buys: 1844


#Applying RNN model

Importing the packages

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import tensorflow as tf
import time

Deciding parameters for the training of the RNN

In [0]:
EPOCHS=10
BATCH_SIZE=64
NAME=f'{SEQ_LEN}--SEQ--{FUTURE_PERIOD_PREDICT}--PRED--{int(time.time())}'

Initializing the  model

In [0]:
model=Sequential()

Adding Recurrent layers 

In [45]:
X_train.shape

(69400, 60, 8)

1st Recurrent Layer or the input layer

In [0]:
model.add(LSTM(128,input_shape=(X_train.shape[1:]),activation='tanh',return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

2nd Recurrent layer

In [0]:
model.add(LSTM(128,activation='tanh',return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

3rd Recurrent layer

In [0]:
model.add(LSTM(128,activation='tanh'))
model.add(Dropout(0.1))
model.add(BatchNormalization())

Adding Dense layers 

In [0]:
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

Adding output layer

In [0]:
model.add(Dense(2,activation='softmax'))

Making the optimizer , to apply SGD and optimize the weights of the network

In [0]:
optimizer=tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)

Compiling the NN (Putting all the information together)

In [0]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [0]:
y_train=np.asarray(y_train)
y_validation=np.asarray(y_validation)

Fitting the model 

In [54]:
history=model.fit(X_train,y_train,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_data=(X_validation,y_validation))

Epoch 1/10
1085/1085 [==============================] - 17s 15ms/step - loss: 0.7223 - accuracy: 0.5139 - val_loss: 0.6969 - val_accuracy: 0.5095
Epoch 2/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6888 - accuracy: 0.5407 - val_loss: 0.6847 - val_accuracy: 0.5393
Epoch 3/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6847 - accuracy: 0.5555 - val_loss: 0.6861 - val_accuracy: 0.5434
Epoch 4/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6830 - accuracy: 0.5596 - val_loss: 0.6752 - val_accuracy: 0.5770
Epoch 5/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6817 - accuracy: 0.5635 - val_loss: 0.6805 - val_accuracy: 0.5700
Epoch 6/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6802 - accuracy: 0.5673 - val_loss: 0.6760 - val_accuracy: 0.5705
Epoch 7/10
1085/1085 [==============================] - 16s 15ms/step - loss: 0.6788 - accuracy: 0.5690 - val_loss: 0.6728 -

### Notes:



1.   For the recurrent layers some activation functions work better and faster ,in this case the 'tanh' function works better than the 'relu' function.

2. We can further tweek the model to improve the accuracy , make it faster , less complex and also avoid overfitting.

3. Historical data is not always the best predictor of the future trends so beware of the predictions.

4. While preprocessing data , it is really important to check and keep an eye out for bugs. In projects like these it is very easy to loose track and create blunders.

